# Bert Regressor

In [ ]:
import tensorflow as tf
import modeling
import tokenization

import run_regressor

from sklearn.utils import shuffle

In [ ]:
bert_config_file = '/home/ubuntu/bert_models/uncased_L-12_H-768_A-12/bert_config.json'
bert_config = modeling.BertConfig.from_json_file(bert_config_file)
vocab_file = '/home/ubuntu/bert_models/uncased_L-12_H-768_A-12/vocab.txt'

max_seq_length = 128

In [ ]:
class BERT_REGRESSOR(object):
    
    def __init__(self,bert_config_file,max_seq_length,is_training):
        self.bert_config_file = bert_config_file
        self.bert_config = modeling.BertConfig.from_json_file(bert_config_file)
        self.max_seq_length = max_seq_length
        
        ''' 
            Computation Graph
        '''
        input_ids = tf.placeholder(tf.int32,[None,max_seq_length],name='input_ids')
        input_mask = tf.placeholder(tf.int32,[None,max_seq_length],name='input_mask')
        segment_ids = tf.placeholder(tf.int32,[None,max_seq_length],name='segment_ids')
        values = tf.placeholder(tf.float32,[None],name='values')
        
        model = modeling.BertModel(config=self.bert_config,
                                   is_training=is_training,
                                   input_ids=input_ids,
                                   input_mask=input_mask,
                                   token_type_ids=segment_ids,
                                   use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        
        hidden_size = output_layer.shape[-1].value
        
        output_weights = tf.get_variable(
          "output_weights", [1, hidden_size],
          initializer=tf.truncated_normal_initializer(stddev=0.02))

        output_bias = tf.get_variable(
          "output_bias", [1], initializer=tf.zeros_initializer())
        
        with tf.variable_scope("loss"):
            if is_training:
                # I.e., 0.1 dropout
                output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

            logits = tf.matmul(output_layer, output_weights, transpose_b=True)
            logits = tf.nn.bias_add(logits, output_bias)
            logits = tf.squeeze(logits)

            loss = tf.losses.mean_squared_error(values,logits)
            self.loss = loss

In [ ]:
class Model_HANDLER(object):
    
    def __init__(self,processor,data_dir,vocab_file,max_seq_length):
        self.processor = processor
        self.tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file,do_lower_case=True)
        self.data_dir = data_dir
        self.max_seq_length = max_seq_length
    
    def make_datasets(self,get_example_fn):
        examples = get_example_fn(self.data_dir)
        features = run_regressor.convert_examples_to_features(examples,
                                                              max_seq_length=self.max_seq_length,
                                                              tokenizer=self.tokenizer)
        input_ids = []
        input_mask = []
        segment_ids = []
        values = []
        for feature in test_features:
            input_ids.append(feature.input_ids)
            input_mask.append(feature.input_mask)
            segment_ids.append(feature.segment_ids)
            values.append(feature.label_id)
        
        input_ids_tensor = tf.constant(input_ids)
        input_mask_tensor = tf.constant(input_mask)
        segment_ids_tensor = tf.constant(segment_ids)
        values_tensor = tf.constant(values)
        
        datasets = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(input_ids_tensor),
                                        tf.data.Dataset.from_tensor_slices(input_mask_tensor),
                                        tf.data.Dataset.from_tensor_slices(segment_ids_tensor),
                                        tf.data.Dataset.from_tensor_slices(label_ids_tensor)))
        return datasets
        
    
    def fit():
        

In [ ]:
processor = run_regressor.ArdProcessor()

In [ ]:
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file,do_lower_case=True)

In [ ]:
test_examples = processor.get_test_examples('/home/ubuntu/glue_data/ARD/')

In [ ]:
test_features = run_regressor.convert_examples_to_features(test_examples,
                                                            max_seq_length=max_seq_length,
                                                            tokenizer=tokenizer)

In [ ]:
all_input_ids = []
all_input_mask = []
all_segment_ids = []
all_label_ids = []

In [ ]:
for feature in test_features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)


In [ ]:
input_ids=tf.constant(all_input_ids[:50])
input_mask=tf.constant(all_input_mask[:50])
segment_ids=tf.constant(all_segment_ids[:50])
label_ids = tf.constant(all_label_ids[:50])

In [ ]:
datasets = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(input_ids),
                                tf.data.Dataset.from_tensor_slices(input_mask),
                                tf.data.Dataset.from_tensor_slices(segment_ids),
                                tf.data.Dataset.from_tensor_slices(label_ids)))

In [ ]:
datasets.output_types

In [ ]:
datasets.output_shapes

In [ ]:
tf.estimator.Estimator

In [ ]:
 = datasets.batch(batch_size=10)

In [ ]:
batch

In [ ]:
model = modeling.BertModel(
  config=bert_config,
  is_training=True,
  input_ids=input_ids,
  input_mask=input_mask,
  token_type_ids=segment_ids,
  use_one_hot_embeddings=False)

In [ ]:
num_labels = 1
is_training = True

In [ ]:
output_layer = model.get_pooled_output()

hidden_size = output_layer.shape[-1].value

output_weights = tf.get_variable(
  "output_weights", [1, hidden_size],
  initializer=tf.truncated_normal_initializer(stddev=0.02))

output_bias = tf.get_variable(
  "output_bias", [num_labels], initializer=tf.zeros_initializer())

with tf.variable_scope("loss"):
    if is_training:
        # I.e., 0.1 dropout
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    #probabilities = tf.nn.softmax(logits, axis=-1)
    #log_probs = tf.nn.log_softmax(logits, axis=-1)
    logits = tf.squeeze(logits)

    loss = tf.losses.mean_squared_error(label_ids,logits)


In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
loss_ = loss.eval()
sess.close()

In [ ]:
loss_